# Oral microbiome transplant workflow

Sonia Nath
Date- 01/04/2023

Oral microbiome transplantation is a novel concept of tranferring microbiota from a healthy donor to a recipient with oral disease. The donor should have a stable, diverse and abundant and healthy microbiome that would result in oral health outcomes in the recipeient host with dental caries. 
This describes the QIIME2 bioinformatics workflow for analysis. 

We analyse the data we used Quantitative Insight Microbial Ecology version 2 bioinformatics platform.
qiime2-2022.11 version

*Special notes about the dataset:*
There were two sequencing runs. June_run and November_run. Both the files need to be merged together. 
November_run has 82 samples, 10 EBCs, 17 washes, and 14 PCR. Total files 123.
June run has 9 samples, 4 EBC and 3 PCR controls. =Total  16
Combined samples size is 82+9 = 91 samples Total files 123+16=139

***

## Step 1. Copying and making a directory

Make a new directory.  

In [ ]:
cd /a1799090/November_run
mkdir run_copy_november

Create a .txt file for samples to copy---- sample_list_to_copy.txt and explore the file. The head command will give the first 10 in the list.

In [ ]:
head sample_list_to_copy.txt

To explore the folder, the first command is to see the folder with the sizr and the second command gives only the names. 

In [ ]:
ls -al
ls -a

The run_1 folder has all the files. Copying files from run_1 to run_copy

In [ ]:
for file in $(cat sample_list_to_copy.txt) ; do cp /a1799090/November_run/run_1/"$file"*.fastq.gz run_copy ; done

In [ ]:
cd run_copy
ls -a
ls -al
ls -lh
ls -alh #some weird files at the beginning of the list
ls -alh | wc -l  # counts the files
ls -alh | head  # gave a count of 250, counting the weird files in the beginning. 
ls -1 | wc -l  #gave a result of 246, which is all the 123 files

<div class="alert alert-block alert-info">**Troubleshooting Notes**:
ls -al 9112022PCRneg*   # to see if the names are correct in the files
cp 9112022PCRneg* ../run_copy   ## to copy individual files to run_copy

Now for June_run

cd a1799090/June_run/
Files present in the June_run include sample-metadata_june.xlsx, sample_list_to_copy, 
run_1 (folder has all the fast q files from June).
Make directory run_copy_june

In [ ]:
mkdir run_copy_june
head sample_list_to_copy.txt

In [ ]:
for file in $(cat sample_list_to_copy.txt) ; do cp a1799090/June_run/run_1/*"$file"*.fastq.gz run_copy_june ; done

In [ ]:
cd a1799090/June_run/run_copy_june
ls -al
ls -alh
ls -alh | head
ls -1 | wc -l 

**Troubleshooting Notes**
In case the copy function didn't work, then may need to copy code. 
cd a1799090/June_run/run_copy_june
cp SE18PLQPBS169Single* ../run_copy_june

also if the loop doesn't work follow the steps to install homebrew.
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
OR
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

Then install dos2unix
brew install dos2unix
dos2unix sample_list_to_copy.txt

Then use the above loop.

Merging Folders June_run and November_run

In [ ]:
cp run_copy_june a1799090/November_run/run_copy_june
###Go to November_run folder
cd /Users/spn5456/November_run
###Now create a directory run_all that will contain all the files
mkdir run_all
cp -fr run_copy_June/ run_all/ && cp -fr run_copy_november/ run_all/
ls -1 | wc -l

The run_all folder should have 278 total files   139*2 (one forward and reverse read)

In [ ]:
## To see all the previous commands
fc -l 1 100
fc -1 | grep ssh

***

## Step 2. QIIME2 analysis

### A. Data import

Make a new directory to store all qiime analysis

In [ ]:
cd
ls
mkdir qiime2analysis
cd qiime2analaysis

In [ ]:
#activate qiime
conda activate qiime2-2022.11

Run the command to import the raw data from run_copy and export it to a single Qiime2 artefact file, demux-paired-end.qza. Use the run_all file that contain all the paired fastq files. 

In [ ]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path run_all \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

The name was long and changed name demux-paired-end.qza to demux.qza

In [ ]:
!mv demux-paired-end.qza demux.qza

### B. Visualising the sequence quality

To summarise the output. The demux summarise provides with a visual information of the distribution of sequence qualities at each position in the sequence data for the next step in the pipeline. 

In [ ]:
!qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv

 Things to look for 1. Where does the median quality drop below 30. 2. Do any of the samples have only few sequences i.e., less than 1000. If so, I would want to omit them from the analysis. 

In [ ]:
!qiime tools view demux.qzv ###directly opens qiime2 viewer

The forward sequence looks like 230 and reverse looks like 200, going with 200. Setting the threshold to 30.

The next step is to clean the data according to sequence error sung the deblur.

In [ ]:
!qiime quality-filter q-score \
    --i-demux demux.qza \
    --o-filtered-sequences dumux-filtered.qza \ ## used in the next step
    --o-filter-stats demux-filter-stats.qza ## we will visualise this file later on 

Files used from the quality filter q score creating .qzv files
assess the output data. The second output.

In [ ]:
!qiime metadata tabulate \
  --m-input-file demux-filter-stats.qza \
  --o-visualization demux-filter-stats.qzv

!qiime tools view demux-filter-stats.qzv

Lots of samples had low read less than 10. 
This samples were removed in the next steps
11282022PCRNeg1 12052022PCRNeg  12142022PCRNeg 12162022PCRneg1 MS53PLQ0610 MS76PLQ0710 MS86PLQ0710 MS93PLQ0710 MS98PLQ0710 MS99PLQ0710 MS107PLQ0610 MS108PLQ0610 MS171PLQ1010 MS177PLQ1010 MS189PLQ0710 MS191PLQ0610 MS203PLQ1010 MS206PLQ1010 MS219PLQ0610 MS223PLQ0610 MS244PLQ0610

### C. Denoising sequences, and feature table construction

We used deblur to denoise-paired ends. This method allows to remove low quality regions of the sequence.

In [ ]:
!qiime deblur denoise-16S \
 --i-demultiplexed-seqs dumux-filtered.qza \
 --p-trim-length 200 \--p-sample-stats \
 --o-representative-sequences representative-sequences.qza \
 --o-table table.qza \
 --o-stats denoising-stats.qza

Three output generated:
1. The table.qza is the feature table file. This is FeatureTable[Frequency] QIIME2 artefact that contain counts (frequencines ) of each unique sequences in each of the OMT dataset. 
2. The name was changed of epresentative-sequences.qza rep-seqs.qza. The rep-seq is a FeatureData[Sequence] QIIME2 artefact that maps feature identifier in the Featuretable to the sequences they represent. 
3. And the third output denoising stats

In [ ]:
!mv representative-sequences.qza rep-seqs.qza

### D. Visualising all the files

In [ ]:
!qiime deblur visualize-stats \
  --i-deblur-stats denoising-stats.qza \
  --o-visualization deblur-stats.qzv

In [ ]:
!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

The visualisation graphs shoes that some samples were filtered out in the denoising step. From 139 reduced to 117 samples
###### *21 samples were filtered. Biological samples were 74; control wash 19; control_PCR 13; control EBC 11*

In [ ]:
!qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

## Step 3. Building a phylogenetic tree

A phylogenetic tree is created on order to generate phylogenetic diversity metrics, such as alpha and beta diversity. A rooted phylogentic tree is created which relates the feature to one another. 
To generate a phylogenetic tree of representative sequences the first is the alignment of representative sequnces.

In [ ]:
!qiime alignment mafft \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza

To mask position that are highly variable (non-biological)

In [ ]:
!qiime alignment mask \
  --i-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza

Use fastree to make the alignment into a tree

In [ ]:
!qiime phylogeny fasttree \
  --i-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza

The last step is to root the tree

In [ ]:
!qiime phylogeny midpoint-root \
  --i-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

## Step 4. Taxonomic analysis

### A. Assign taxonomy

This step is to identify the taxonomies in the samples. The ASVs are limited and identifying the bacterial strains would be more useful. It is to put a name to the ASVs. 
This is done in two steps. First we need a  reference database, and second we need an alogorithm for identifying these sequences using the database. 

Before starting we need to download the taxonomic classifer from here:
https://docs.qiime2.org/2022.8/data-resources/
downloaded the silva 138 99% OTU full-length sequence

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier silva-138-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

In [ ]:
!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

!qiime tools view taxanomy.qzv

### B. Exploring the dataset. 

The data set was ecplored to see the differences between the biological samples and control. We had three types of controls in the dataset; the curette washes, extraction blank control (EBC) and PCR controls. 

Building taxabar plots for visualisation to see differences in samples and control. Use control-(Y/N) metacolumn in the sample metadata sheet to observe the differences. 
Alpha and beta diversity plots are used to observe the differences

In [ ]:
!qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file sample-metadata.txt \
  --o-visualization Taxonomy_BarPlots.qzv

In [ ]:
!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

In [ ]:
 !qiime diversity core-metrics \
  --i-table table.qza \
  --p-sampling-depth 2000 \
  --m-metadata-file sample-metadata.txt \
  --output-dir CoreDiversity_Results

In [ ]:
!qiime diversity core-metrics \
  --i-table table.qza \
  --p-sampling-depth 700 \
  --m-metadata-file sample-metadata.txt \
  --output-dir CoreDiversity_Results_tree \
  --tree rooted-tree.qza

In [ ]:
!qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --p-max-depth 2000 \
  --m-metadata-file sample-metadata.txt \
  --o-visualization CoreDiversity_Results/alpha_rarefaction.qzv

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity CoreDiversity_Results/observed_features_vector.qza \
  --m-metadata-file sample-metadata.txt \
  --o-visualization CoreDiversity_Results/observed_features.qzv

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity CoreDiversity_Results/shannon_vector.qza \
  --m-metadata-file sample-metadata.txt \
  --o-visualization CoreDiversity_Results/shannon_vector_significance.qzv

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix CoreDiversity_Results/bray_curtis_distance_matrix.qza \
  --m-metadata-column Control \
  --m-metadata-file sample-metadata.txt \
  --p-pairwise \
  --o-visualization CoreDiversity_Results/bray_curtis_significance_Control.qzv

 #### Using Aitchinson Distance Matrix (AM) for Taxonomic data 
 pip install deicode

In [ ]:
!mkdir CoreDiversity_Results/Aitchinson

In [ ]:
# Create AM with Deicode tool
!qiime deicode rpca \
    --i-table table.qza \
    --p-min-feature-count 10 \
    --o-biplot CoreDiversity_Results/Aitchinson/AM-ordination.qza \
    --o-distance-matrix CoreDiversity_Results/Aitchinson/aitchison-distance.qza 

In [ ]:
#Plot biplot with 3 features 
!qiime emperor biplot \
    --i-biplot CoreDiversity_Results/Aitchinson/AM-ordination.qza \
    --m-sample-metadata-file sample-metadata.txt \
    --o-visualization CoreDiversity_Results/Aitchinson/biplot-3-features.qzv \
    --p-number-of-features 3 

In [ ]:
# Plot biplot with 10 features
!qiime emperor biplot \
    --i-biplot CoreDiverisy_Results/Aitchinson/AM-ordination.qza \
    --m-sample-metadata-file sample-metadata.txt \
    --o-visualization CoreDiversity_Results/Aitchinson/biplot-10-features.qzv \
    --p-number-of-features 10

Changing the files for Genus level, to names are shown instead of taxonomic ids

In [ ]:
!qiime taxa collapse \
--i-table table.qza \
--i-taxonomy taxonomy.qza \
--p-level 6 \
--o-collapsed-table table_genus.qza

In [ ]:
!qiime feature-table summarize \
  --i-table table_genus.qza \
  --o-visualization table_genus.qzv \
  --m-sample-metadata-file sample-metadata.txt

Repeating the steps again. Instead for feature name it will now show the genus name

In [ ]:
# Create AM with Deicode tool
!qiime deicode rpca \
    --i-table table_genus.qza \
    --p-min-feature-count 10 \
    --o-biplot CoreDiversity_Results/Aitchinson/AM-ordination-genus.qza \
    --o-distance-matrix CoreDiversity_Results/Aitchinson/aitchison-distance-genus.qza 

# Plot biplot with 3 features 
!qiime emperor biplot \
    --i-biplot CoreDiversity_Results/Aitchinson/AM-ordination-genus.qza \
    --m-sample-metadata-file sample-metadata.txt \
    --o-visualization CoreDiversity_Results/Aitchinson/biplot-3-features-genus.qzv \
    --p-number-of-features 3 

#### Beta diversity group significance tests with AM 

In [ ]:
# Beta diversity group significance (Aitchinson): Caries
!qiime diversity beta-group-significance \
  --i-distance-matrix CoreDiversity_Results/Aitchinson/aitchison-distance.qza \
  --m-metadata-file sample-metadata.txt \
  --m-metadata-column SampleORControl \
  --o-visualization CoreDiversity_Results/BetaSig-PNOVA/vis-AM-SOC-significance.qzv \
  --p-pairwise 

### All the plots looks good!!
We could see clear differences in the sample and controls in the alpha diversity plots. 
The beta diversity; the bray curtis showed clear cluster separation of controls and biological samples. All the controls clutered together. 
There were few curette washes (5) that clustered together with the washes. 

## Step 5. Filtering and decontamination steps

We need to use decontam in R in the next step to filter out the contaminants present.
The decontamination would be done in two steps
A. First only the real controls, ie.EBC and PCR
B. Then the curette wash control

### A.1. Filtering the samples and the controls (EBC and PCR) 

Using the filtering function in QIIME2

In [ ]:
!qiime feature-table filter-samples \
--i-table table.qza \
--m-metadata-file sample-metadata.tsv \
--p-where "[SampleORControl] IN ('sample', 'control_EBC', 'control_PCR')" \
--o-filtered-table table-filtered-controls-EBC-PCR.qza    

#### view the table ###
!qiime feature-table summarize \
--i-table table-filtered-controls-EBC-PCR.qza \
--o-visualization table-controls-EBC-PCR.qzv \
--m-sample-metadata-file sample-metadata.tsv

!qiime tools view table-filtered-controls-EBC-PCR.qzv

Note: The table summary showed Biological samples were 74;  control_PCR 13; control EBC 13. In total 100 samples

### A.2. Go to R for decontam (Filter step 1)

Use the sammple-metadata file, and table-filtered-controls-EBC-PCR.qza in decontam

Run the script for decontam and this will generate a contaminant list. 
contaminant_list.tsv

In [ ]:
cp contaminant_list.tsv contaminant_list-controls-EBC-PCR.tsv

### A.3.Filter out the contaminants from PCR and EBC

In [ ]:
!qiime feature-table filter-features \
  --i-table table.qza \
  --p-exclude-ids \
  --m-metadata-file contaminants_list-controls-EBC-PCR.tsv \
  --o-filtered-table decontaminated-controls-EBC-PCR.qza
  
## Visualise results from post-decontamination to cross-check that results are consistent with results from R.
!qiime feature-table summarize \
  --i-table decontaminated-controls-EBC-PCR.qza \
  --m-sample-metadata-file sample-metadata.tsv \
  --o-visualization decontaminated-controls-EBC-PCR.qzv

##### 978 ASVs features remaining; initially the raw file had 1004, and decontam detected 26 contaminants. 1004-978= 26
Note: Biological samples were 74 (unchanged); control wash 16 (17); control_PCR 11 (13); control EBC 13 (13)

Explore the beta diversity. The decontaminated file which has samples, PCR and controls AFTER DECONTAMINATION without removing the controls.

In [ ]:
!mkdir qiime-analysis-decontam-EBC-PCR

In [ ]:
!qiime diversity core-metrics \
  --i-table decontaminated-controls-EBC-PCR.qza \
  --p-sampling-depth 1000 \
  --m-metadata-file sample-metadata.tsv \
  --output-dir qiime-analyis-decontam-EBC-PCR/CoreDiversity_Results

### B.1 Filtering samples and control_wash

 Filtering out the samples, and curette wash

In [ ]:
qiime feature-table filter-samples \
--i-table table.qza \
--m-metadata-file sample-metadata.tsv \
--p-where "[SampleORControl] IN ('sample', 'control_wsh')" \
--o-filtered-table table-filtered-controls-wash.qza

##visualise the table
!qiime feature-table summarize \
--i-table table-filtered-controls-wash.qza \
--o-visualization table-filtered-controls-wash.qzv \
--m-sample-metadata-file sample-metadata.tsv

!qiime tools view table-filtered-controls-wash.qzv

Note: Total samples 91, samples 74, and control_wsh 17

### B.2. Go to R for decontam (Filter step 2)

This is second step to generating the contaminants that are present in the curette washes, control_wsh. 

Run the script in R and this generate a contaminant list.

In [ ]:
!cp contaminant_list.tsv contaminant_list-controls-EBC-PCR.tsv

### B.3. Filter out the contaminants from control_wsh

Remove/filter out contaminants.The contaminants in the wash; use the file that was filtered out for contaminants present in the PCR and EBC. 

In [ ]:
!qiime feature-table filter-features \
  --i-table decontaminated-controls-EBC-PCR.qza \
  --p-exclude-ids \
  --m-metadata-file contaminants_list-controls-wash.tsv \
  --o-filtered-table decontaminated-controls-wash.qza

## Visualise results post-decontamination to cross-check that results are consistent with results from R.
!qiime feature-table summarize \
  --i-table decontaminated-controls-wash.qza \
  --m-sample-metadata-file sample-metadata.tsv \
  --o-visualization decontaminated-controls-wash.qzv

*Note:* 901 ASVs features remaining; the post removal of EBC and PCR had 978, and decontam detected 89 contaminants.
  Samples 74; control_wash 16 (17) control PCR 10 (13) control EBC 12

Explore using core metrics the decontaminated-controls-wash. look at beta diversity.
Exploring decontaminated file which has samples, and controls AFTER 2 step DECONTAM without removing the controls

In [ ]:
 !qiime diversity core-metrics \
  --i-table decontaminated-controls-wash.qza \
  --p-sampling-depth 1000 \
  --m-metadata-file sample-metadata.tsv \
  --output-dir CoreDiversity_Results-control-wash 

After the completetion of decontamination twice we can remove the controls from the feature table and that will be out final table for downstram analysis

## Step 6. Removing the controls

Filtering out all the controls

In [ ]:
!qiime feature-table filter-samples \
  --i-table decontaminated-controls-wash.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-where "[Control] IN ('N')" \
  --o-filtered-table samples-clean.qza

Removing samples that were plaque ASM

In [ ]:
!qiime feature-table filter-samples \
  --i-table samples-clean.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-where "NOT [SampleType]='PLAQUE_ASM'" \
  --o-filtered-table sample-clean-filtered.qza

Filtering out samples that were less than 10

In [ ]:
 !qiime feature-table filter-features \
  --i-table sample-clean-filtered.qza \
  --p-min-frequency 10 \
  --o-filtered-table sample-clean-feature-frequency-filtered-table.qza

In [ ]:
!qiime feature-table summarize \
--i-table sample-clean-feature-frequency-filtered-table.qza \
--m-sample-metadata-file sample-metadata.tsv \
--o-visualization sample-clean-feature-frequency-filtered-table.qzv

</b> #### We can start analysing the data for downstream analysis. 